In [1]:
import os
from importlib import reload
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import math
import datetime
import pdblp
import runpy
import QuantLib as ql
import matplotlib as mpl
#import matplotlib
mpl.get_backend()
import matplotlib.pyplot as plt
plt.interactive(False)
import seaborn as sns
from tabulate import tabulate
from sklearn.preprocessing import minmax_scale
from sklearn.mixture import GaussianMixture
from scipy.stats import zscore
import pickle
import re
import concurrent.futures
import time
from datetime import datetime, timedelta, time
from scipy import stats
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:,}'.format
pd.options.mode.chained_assignment = None

## BBG API
con = pdblp.BCon(debug=False, port=8194, timeout=50000)
con.start()

from Utilities import *
from Conventions import FUT_CT, FUT_CT_Q, ccy, ccy_infl, hist
from OIS_DC_BUILD import ois_dc_build, get_wirp, ois_from_nodes
from SWAP_BUILD import swap_build, libor_from_nodes
from SWAP_PRICER import Swap_Pricer, Swap_curve_fwd, quick_swap
from SWAP_TABLE import swap_table, swap_table2, curve_hmap
from INF_ZC_BUILD import infl_zc_swap_build, Infl_ZC_Pricer, inf_swap_table
from BOND_CURVES import bond_curve_build
from VOL_BUILD import build_vol_surf, build_vol_spline, bond_fut_opt_strat, get_sim_option_px, build_stir_vol_surf, stir_opt_strat
from PLOT import plt_curve, plt_inf_curve, plt_opt_strat, rates_hm, curve_hm, plt_ois_curve, plot_opt_vol_surf, plt_stir_opt_strat, plotool, ecfc_plot

# bokeh imports
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, BoxZoomTool, ResetTool, Legend, DatetimeTickFormatter, SingleIntervalTicker, LinearAxis, FuncTickFormatter
from bokeh.models import ColumnDataSource, TabPanel, Tabs, LabelSet, Span, Range1d, FactorRange, Range, Label, LinearColorMapper
from bokeh.layouts import row, column, gridplot, layout
from bokeh.palettes import Category10, brewer, Category20, Bright6, Blues8, Blues4, Cividis256
from bokeh.transform import factor_cmap
from bokeh.models.renderers import GlyphRenderer
from itertools import accumulate
from bokeh.colors import RGB
from matplotlib import cm

In [7]:

linkers = ['GB00BYY5F144', 'GB00B128DH60', 'GB00BZ1NTB69', 'GB00B3Y1JG82', 'GB0008932666', 'GB00BNNGP551', 'GB00B3D4VD98', 'GB00BMF9LJ15', 'GB00B46CGH68', 'GB0031790826', 'GB00BYZW3J87',
           'GB00B1L6W962', 'GB00BLH38265', 'GB00B3LZBF68', 'GB00BGDYHF49', 'GB00B3MYD345', 'GB00B7RN0G65', 'GB00BMF9LH90', 'GB00BYMWG366', 'GB00B24FFM16', 'GB00BZ13DV40', 'GB00B421JZ66',
           'GB00BNNGP882', 'GB00B73ZYW09', 'GB00B0CNHZ09', 'GB00BYVP4K94', 'GB00BP9DLZ64', 'GB00B4PTCY75', 'GB00BD9MZZ71', 'GB00BDX8CX86', 'GB00BM8Z2W66']

comparators = ['GB00BL68HJ26', 'GB00BDRHNP05', 'GB00BMBL1G81', 'GB00BLPK7227', 'GB00B24FF097', 'GB0004893086', 'GB0004893086', 'GB0004893086', 'GB00B52WS153', 'GB0032452392', 'GB0032452392',
               'GB00BZB26Y51', 'GB00BJQWYH73', 'GB00BJQWYH73', 'GB00BJQWYH73', 'GB00B1VWPJ53', 'GB00B84Z9V04', 'GB00BNNGP775', 'GB00BDCHBW80', 'GB00BDCHBW80', 'GB00BMBL1F74', 'GB00BMBL1F74',
               'GB00BLH38158', 'GB00B6RNH572', 'GB00B06YGN05', 'GB00BD0XH204', 'GB00B54QLM75', 'GB00BMBL1D50', 'GB00BYYMZX75', 'GB00BFMCN652', 'GB00BLBDX619']



linkers = [linkers[i] + ' Govt' for i in np.arange(len(linkers))]
comparators = [comparators[i] + ' Govt' for i in np.arange(len(comparators))]

today = ql.Date(datetime.datetime.now().day, datetime.datetime.now().month, datetime.datetime.now().year)
start_dt = bbg_date_str(ql.UnitedKingdom().advance(today, ql.Period('-1Y')))
latest = bbg_date_str(today)

linker_feed = list(np.array(linkers)[[5,10,15]])
feed = [linkers.index(linker_feed[i]) for i in np.arange(len(linker_feed))]
comp_feed = list(np.array(comparators)[feed])


df1 = con.bdh( linker_feed, ['YLD_YTM_MID', 'Z_SPRD_MID'], start_dt, latest)
df2 = con.bdh( comp_feed, ['YLD_YTM_MID', 'Z_SPRD_MID'], start_dt, latest)

ry = get_linker_metrics(df1, linker_feed, m='fly')
nom = get_linker_metrics(df2, comp_feed, m='fly')

comb = pd.DataFrame()
comb['fly'] = nom['fly'] - ry['fly']
comb['z_sprd'] = nom['flyz_sprd'] - ry['flyz_sprd']
comb['z_score_1m'] = roll_zscore(comb['fly'], 20)
comb['z_score_3m'] = roll_zscore(comb['fly'], 60)
comb['z_score_6m'] = roll_zscore(comb['fly'], 180)

comb['rel_z_z_score_1m'] = roll_zscore(comb['z_sprd'], 20)
comb['rel_z_z_score_3m'] = roll_zscore(comb['z_sprd'], 60)
comb['rel_z_z_score_6m'] = roll_zscore(comb['z_sprd'], 180)




In [8]:
comb

,fly,z_sprd,z_score_1m,z_score_3m,z_score_6m,rel_z_z_score_1m,rel_z_z_score_3m,rel_z_z_score_6m
0,-1.200000000000001,4.651000000000001,NaN,NaN,NaN,NaN,NaN,NaN
1,-3.7000000000000366,2.4239999999999995,NaN,NaN,NaN,NaN,NaN,NaN
2,-5.600000000000049,1.4179999999999993,NaN,NaN,NaN,NaN,NaN,NaN
3,-5.499999999999994,1.258,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.5999999999999792,4.679999999999999,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
772,-13.700000000000001,2.999999999999993,0.5798730004531703,0.7749378792049656,-0.7013598248257789,0.5552783721885529,0.12601254783929827,-0.2890243623219565
773,-14.100000000000023,2.4199999999999875,0.4572271739278077,0.5502446510187725,-0.7737879857636367,0.3406659965031252,-0.14979881680542775,-0.4849392648407529
774,-11.999999999999966,7.049999999999997,2.098249213313573,1.6763700548830687,-0.335777030892035,3.855092443356965,1.96279922866612,1.0027530031186065
775,-11.89999999999991,6.1980000000000075,1.8784455273512155,1.6605024171062135,-0.3062929634546851,2.358701659650999,1.4907726817599627,0.7167182569203951


In [13]:
s_plot = []
#s1_source = ColumnDataSource(data=dict(x=fix_months, y=fixings, z=fixings_chg, b=barcap_f ))

s1 = figure(title = 'Yield', width=550, height=300, tools=["pan", "tap", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s1.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
s1.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y{0.0}')], formatters={'$x': 'datetime'}))
s1.xgrid.visible = False
s1.ygrid.visible = False
s1.line(x= ry['date'], y=ry['fly'], line_width=0.9, color='lightsteelblue', muted_alpha=0.1, legend_label="RY Fly")
s1.line(x= nom['date'], y=nom['fly'], line_width=0.9, color='sandybrown', muted_alpha=0.1, legend_label="Nom Fly")
s1.xaxis.visible = False
s1.legend.location = 'top_left'
s1.legend.label_text_font = "calibri"
s1.legend.label_text_font_size = "9pt"
s1.legend.spacing = 1
s1.legend.click_policy = "mute"
s1.legend.background_fill_alpha = 0.0


s2 = figure(title = 'Spread', width=550, height=300, tools=["pan", "tap", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s2.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
s2.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y{0.0}')], formatters={'$x': 'datetime'}))
s2.xgrid.visible = False
s2.ygrid.visible = False
s2.line(x=ry['date'], y=comb['fly'], line_width=0.7, color='firebrick', alpha=0.8)
s2.xaxis.major_label_orientation = math.pi / 2


s3 = figure(title = 'Z-Scores Relative', width=550, height=200, tools=["pan", "tap", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s3.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
s3.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y{0.0}')], formatters={'$x': 'datetime'}))
s3.xgrid.visible = False
s3.ygrid.visible = False
s3.line(x=ry['date'], y=comb['z_score_3m'], line_width=0.6, color='indigo', alpha=0.8, legend_label="z_score: 3m")
s3.line(x=ry['date'], y=comb['z_score_6m'], line_width=0.6, color='green', alpha=0.8, legend_label="z_score: 6m")
zero_line = Span(location=0, dimension='width', line_color='darkslategray', line_width=1)
s3.renderers.extend([zero_line])
s3.xaxis.visible = False
s3.legend.location = 'top_left'
s3.legend.label_text_font = "calibri"
s3.legend.label_text_font_size = "9pt"
s3.legend.spacing = 1
s3.legend.click_policy = "mute"
s3.legend.background_fill_alpha = 0.0


s4 = figure(title = 'Z-Spread', width=550, height=300, tools=["pan", "tap", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s4.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
s4.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y{0.0}')], formatters={'$x': 'datetime'}))
s4.xgrid.visible = False
s4.ygrid.visible = False
s4.line(x= ry['date'], y=ry['flyz_sprd'], line_width=0.9, color='lightsteelblue', legend_label="UKTi Z-Sprd Fly")
s4.line(x= nom['date'], y=nom['flyz_sprd'], line_width=0.8, color='sandybrown', legend_label="UKT Z-Sprd Fly")
s4.xaxis.visible = False
s4.legend.location = 'top_left'
s4.legend.label_text_font = "calibri"
s4.legend.label_text_font_size = "9pt"
s4.legend.spacing = 1
s4.legend.click_policy = "mute"
s4.legend.background_fill_alpha = 0.0


s5 = figure(title = 'Relative Z-Spread', width=550, height=300, tools=["pan", "tap", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s5.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
s5.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y{0.0}')], formatters={'$x': 'datetime'}))
s5.xgrid.visible = False
s5.ygrid.visible = False
s5.line(x=ry['date'], y=comb['z_sprd'], line_width=0.7, color='firebrick', alpha=0.8)
s5.xaxis.major_label_orientation = math.pi / 2


s6 = figure(title = 'Z-Scores Relative', width=550, height=200, tools=["pan", "tap", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s6.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
s6.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y{0.0}')], formatters={'$x': 'datetime'}))
s6.xgrid.visible = False
s6.ygrid.visible = False
s6.line(x=ry['date'], y=comb['rel_z_z_score_3m'], line_width=0.6, color='indigo', alpha=0.8, muted_alpha=0.1, legend_label="z_score: 3m")
s6.line(x=ry['date'], y=comb['rel_z_z_score_6m'], line_width=0.6, color='green', alpha=0.8, muted_alpha=0.1, legend_label="z_score: 6m")
zero_line = Span(location=0, dimension='width', line_color='darkslategray', line_width=1)
s6.renderers.extend([zero_line])
s6.xaxis.visible = False
s6.legend.location = 'top_left'
s6.legend.label_text_font = "calibri"
s6.legend.label_text_font_size = "9pt"
s6.legend.spacing = 1
s6.legend.click_policy = "mute"
s6.legend.background_fill_alpha = 0.0




p=layout(children=[[s1, s4], [s2, s5], [s3, s6]])
show(p)


#s_plot.append(s1)
#s_plot.append(s2)
#s_plot.append(s3)

#layout = column(*s_plot)
#show(layout)


In [79]:
###### fx flies vs realised
start = '20061201'
today = ql.Date(datetime.datetime.now().day, datetime.datetime.now().month, datetime.datetime.now().year)
latest = bbg_date_str(today)

pair = ['EURUSD', 'GBPUSD', 'USDJPY', 'USDCAD', 'AUDUSD', 'USDCHF', 'USDSEK', 'USDNOK', 'NZDUSD', 'USDBRL', 'USDMXN', 'USDZAR', 'EURCHF','EURHUF', 'EURPLN',
        'EURJPY', 'GBPJPY', 'AUDJPY', 'USDKRW', 'EURSEK', 'EURNOK', 'USDIDR', 'USDCLP', 'AUDNZD', 'USDSGD']
tenor = '6M'
real_tenor = '3M'
bfl_delta = 10

implied_pre = []
realised_post = []
implied_post = []
bfly_pre = []
bfly_post = []

for i in np.arange(len(pair)):
    print(pair[i])
    impl_v_ticker = pair[i]+'V'+tenor+' Curncy'
    bfly_ticker = pair[i]+str(bfl_delta)+'B'+tenor+' Curncy'
    real_v_ticker = pair[i]+'H'+real_tenor+' Curncy'

    implv = con.bdh([impl_v_ticker] , 'PX_LAST', start, latest, longdata = True)
    bflv = con.bdh([bfly_ticker] , 'PX_LAST', start, latest, longdata = True)
    realv = con.bdh([real_v_ticker] , 'PX_LAST', start, latest, longdata = True)

    start = max(bflv['date'][0],implv['date'][0],realv['date'][0]).strftime('%Y%m%d')
    start2 = ql.Date(int(start[6:8]),int(start[4:6]),int(start[:4]))
    bflv['date'] = [datetime_to_ql(bflv['date'][i]) for i in np.arange(len(bflv))]
    implv['date'] = [datetime_to_ql(implv['date'][i]) for i in np.arange(len(implv))]
    realv['date'] = [datetime_to_ql(realv['date'][i]) for i in np.arange(len(realv))]

    bflv = bflv[bflv['date'] >= start2 ]
    implv = implv[implv['date'] >= start2]
    realv = realv[realv['date'] >= start2]
    quantiles = bflv.quantile([.2, .4, 0.6, 0.8])

    for i in np.arange(len(quantiles)+1):
        if i == 0:
            con1 = bflv[bflv['value'] < quantiles.iloc[0].value]['date'].tolist()
        elif i == len(quantiles):
            con1 = bflv[bflv['value'] > quantiles.iloc[-1].value]['date'].tolist()
        else:
            con1 = bflv[ (bflv['value'] < quantiles.iloc[i].value) & (bflv['value'] > quantiles.iloc[i-1].value)  ]['date'].tolist()

        con2 = [con1[i] + ql.Period(real_tenor) for i in np.arange(len(con1))]
        con3 = [con2[i] for i in np.arange(len(con2)) if con2[i] < today]

        implied_pre.append(implv[implv['date'].isin(con1)]['value'].mean())
        bfly_pre.append(bflv[bflv['date'].isin(con1)]['value'].mean())

        realised_post.append(realv[realv['date'].isin(con3)]['value'].mean())
        implied_post.append(implv[implv['date'].isin(con3)]['value'].mean())
        bfly_post.append(bflv[bflv['date'].isin(con3)]['value'].mean())

df = pd.DataFrame()
df['pair'] = flat_lst([[pair[i]]*5 for i in np.arange(len(pair))])
df['implied_tenor'] = [tenor]*len(realised_post)
df['realised_tenor'] = [real_tenor]*len(realised_post)
df['quantile'] = [1,2,3,4,5]*len(pair)
#df['q_value'] = ['<' + str(quantiles['value'].tolist()[0])]+ quantiles['value'].tolist()[1:]+['>' + str(quantiles['value'].tolist()[-1])]
df['impl_pre'] = implied_pre
df['impl_post'] = implied_post
df['imp_chg'] = np.array(implied_post) - np.array(implied_pre)
df['bfl_pre'] = bfly_pre
df['bfl_post'] = bfly_post
df['bfl_chg'] = np.array(bfly_post) - np.array(bfly_pre)
df['real'] = realised_post
df['value'] = np.array(realised_post) - np.array(implied_pre)

df.round(3)

df1 = df[(df['quantile'] == 1) | (df['quantile'] == 5)]
df2 = df1[df1['pair'].isin(['EURUSD','GBPUSD','USDJPY','AUDUSD','USDMXN'])]



EURUSD
GBPUSD
USDJPY
USDCAD
AUDUSD
USDCHF
USDSEK
USDNOK
NZDUSD
USDBRL
USDMXN
USDZAR
EURCHF
EURHUF
EURPLN
EURJPY
GBPJPY
AUDJPY
USDKRW
EURSEK
EURNOK
USDIDR
USDCLP
AUDNZD
USDSGD


In [80]:
df1

,pair,implied_tenor,realised_tenor,quantile,impl_pre,impl_post,imp_chg,bfl_pre,bfl_post,bfl_chg,real,value
0,EURUSD,6M,3M,1,6.313561028571429,6.692728571428572,0.37916754285714305,0.6506766857142858,0.8639162202380952,0.2132395345238094,6.747901339285715,0.434340310714286
4,EURUSD,6M,3M,5,13.767701371428569,12.885451586206896,-0.8822497852216724,2.0657042285714287,1.788450896551724,-0.2772533320197046,11.640440137931035,-2.127261233497533
5,GBPUSD,6M,3M,1,7.103511825487945,7.643728767123288,0.5402169416353431,0.7538838117106773,0.9119412480974125,0.1580574363867352,7.414046270928462,0.310534445440517
9,GBPUSD,6M,3M,5,13.67753444316877,12.277659279778394,-1.3998751633903765,2.0851809414466134,1.7675519390581715,-0.31762900238844183,12.51210886426593,-1.1654255789028412
10,USDJPY,6M,3M,1,7.059592360319271,7.265267727930536,0.20567536761126526,0.9158010262257698,1.1106367583212735,0.19483573209550376,6.566493632416788,-0.493098727902483
14,USDJPY,6M,3M,5,12.44255102739726,12.133149796472185,-0.309401230925074,2.4157183789954333,1.990759972862958,-0.4249584061324754,12.292026322930802,-0.1505247044664575
15,USDCAD,6M,3M,1,6.229998853211009,6.513501335311573,0.2835024821005643,0.5724253440366972,0.7097833827893175,0.13735803875262032,6.834768991097922,0.6047701378869137
19,USDCAD,6M,3M,5,13.08094857142857,11.720090403337968,-1.3608581680906013,1.7722,1.5889986091794162,-0.18320139082058384,11.713471627260084,-1.3674769441684855
20,AUDUSD,6M,3M,1,8.486587285223367,9.115680625931445,0.6290933407080779,0.8153432989690722,1.026967213114754,0.21162391414568182,9.980840238450076,1.4942529532267095
24,AUDUSD,6M,3M,5,16.505179794520547,15.195114902506965,-1.3100648920135818,2.7487157534246576,2.4247040389972145,-0.32401171442744303,15.553380222841227,-0.9517995716793202


In [84]:
s1_source = ColumnDataSource(data=dict(x1=df1['bfl_pre'], x2=df1['bfl_chg'], y1=df1['imp_chg'], z1=df1['real'], z2=df1['value'],
                                       color_schem = ['blue','red']*len(pair), ccy_pair = df1['pair'],  legend_schem = ['20%','80%']*len(pair)))

#s1_source = ColumnDataSource(data=dict(x1=df['bfl_pre'], x2=df['bfl_chg'], y1=df['imp_chg'], z1=df['real'], z2=df['value'], color_schem = ['blue','red','green', 'gold', 'purple']*len(pair),
#                                       ccy_pair = df['pair'],  legend_schem = ['<20%','20-40%', '40-60%','60-80%', '>80%']*len(pair)))

s2_source = ColumnDataSource(data=dict(x1=df2['bfl_pre'], x2=df2['bfl_chg'], y1=df2['imp_chg'], z1=df2['real'], z2=df2['value'], ccy_pair = df2['pair']))

legend_schem = ['20%','80%']*len(pair)

#s1 = figure(title = 'Realised', width=550, height=300, tools=["pan", "tap", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
#s1.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
#s1.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y{0.0}')], formatters={'$x': 'datetime'}))
#s1.xgrid.visible = False
#s1.ygrid.visible = False
#s1.circle(x= 'x1', y='z1', size=5, source=s1_source, color='color_schem', muted_alpha=0.1, legend_label='realised')
#s1.xaxis.visible = True
#s1.legend.location = 'top_right'
#s1.legend.label_text_font = "calibri"
#s1.legend.label_text_font_size = "9pt"
#s1.legend.spacing = 1
#s1.legend.click_policy = "mute"
#s1.legend.background_fill_alpha = 0.0

s2 = figure(title = 'Realised - Ex-Ante Implied', width=800, height=700, tools=["pan", "tap", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s2.toolbar.autohide=True
#s2.add_layout(Legend(), 'right')
#s1.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
#s1.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y{0.0}')], formatters={'$x': 'datetime'}))
s2.xgrid.visible = False
s2.ygrid.visible = False
s2.circle(0, 0, size=0.00000001, color= "#ffffff", legend_label='Flies by Quantiles')
s2.circle(x= 'x1', y='z2', size=5, source=s1_source, color='color_schem', muted_alpha=0.1,  legend_field='legend_schem')
zero_line = Span(location=0, dimension='width', line_color='darkseagreen', line_width=1)
s2.renderers.extend([zero_line])

s2.vspan(x=[0.48],line_width=[1], line_color="red", line_dash='dotted', legend_label = 'eurusd')
s2.vspan(x=[0.495],line_width=[1], line_color="green", line_dash='dotted', legend_label = 'gbpusd')
s2.vspan(x=[0.375],line_width=[1], line_color="blue", line_dash='dotted', legend_label = 'usdcad')

labels_1 = LabelSet(x='x1', y='z2', text='ccy_pair', level='glyph', text_align='center', y_offset=-15, source=s2_source, text_font_size='8px', text_color='black')
s2.add_layout((labels_1))

s2.xaxis.visible = True
s2.xaxis.axis_label = '6m-expiry 10-delta fx flies'
s2.xaxis.axis_label_text_font_size = "10pt"
s2.xaxis.major_label_text_font_size = "8pt"
s2.yaxis.axis_label = 'realised vs ex-ante implied vol diff over next 3m'
s2.yaxis.axis_label_text_font_size = "10pt"
s2.yaxis.major_label_text_font_size = "8pt"
s2.legend.location = 'top_right'
s2.legend.label_text_font = "calibri"
s2.legend.label_text_font_size = "9pt"
s2.legend.spacing = 1
s2.legend.click_policy = "mute"
s2.legend.background_fill_alpha = 0.0

#s3 = figure(title = 'Butterfly Chg', width=500, height=400, tools=["pan", "tap", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
#s1.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
#s1.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y{0.0}')], formatters={'$x': 'datetime'}))
#s3.xgrid.visible = False
#s3.ygrid.visible = False
#s3.circle(0, 0, size=0.00000001, color= "#ffffff", legend_label='Quantiles')
#s3.circle(x= 'x1', y='x2', size=5, source=s1_source, color='color_schem', muted_alpha=0.1, legend_field='legend_schem')
#zero_line = Span(location=0, dimension='width', line_color='darkseagreen', line_width=1)
#s3.renderers.extend([zero_line])
#s3.xaxis.visible = True
#s3.legend.location = 'top_right'
#s3.legend.label_text_font = "calibri"
#s3.legend.label_text_font_size = "9pt"
#s3.legend.spacing = 1
#s3.legend.click_policy = "mute"
#s3.legend.background_fill_alpha = 0.0

s4 = figure(title = 'Implied Vol Chg', width=800, height=700)
s4.toolbar.autohide=True
#s4.add_layout(Legend(), 'right')
#s1.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
#s1.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y{0.0}')], formatters={'$x': 'datetime'}))
s4.xgrid.visible = False
s4.ygrid.visible = False
s4.circle(0, 0, size=0.00000001, color= "#ffffff", legend_label='Flies by Quantiles')
s4.circle(x= 'x1', y='y1', size=5, source=s1_source, color='color_schem', muted_alpha=0.1, legend_field='legend_schem')
zero_line = Span(location=0, dimension='width', line_color='darkseagreen', line_width=1)
s4.renderers.extend([zero_line])

s4.vspan(x=[1.3],line_width=[1], line_color="red", line_dash='dotted', legend_label = 'usdmxn')
s4.vspan(x=[0.63],line_width=[1], line_color="green", line_dash='dotted', legend_label = 'eurchf')

labels_2 = LabelSet(x='x1', y='y1', text='ccy_pair', level='glyph', text_align='center', y_offset=-15, source=s2_source, text_font_size='8px', text_color='black')
s4.add_layout((labels_2))

s4.xaxis.visible = True
s4.xaxis.axis_label = '6m-expiry 10-delta fx flies'
s4.xaxis.axis_label_text_font_size = "10pt"
s4.xaxis.major_label_text_font_size = "8pt"
s4.yaxis.axis_label = 'chg in implied vol over next 3m'
s4.yaxis.axis_label_text_font_size = "10pt"
s4.yaxis.major_label_text_font_size = "8pt"
s4.legend.location = 'top_right'
s4.legend.label_text_font = "calibri"
s4.legend.label_text_font_size = "9pt"
s4.legend.spacing = 1
s4.legend.click_policy = "mute"
s4.legend.background_fill_alpha = 0.0


p=layout(children=[[s2, s4]])
#p=layout(children=[[s2], [s4]])
show(p)



In [55]:
color_schem = ['blue','red']*len(pair)
legend_schem = ['20%','80%']*len(pair)

items = [(color_schem[i], legend_schem[i]) for i in np.arange(len(color_schem))]

#items = []
#for i in range(2):
#    items += [(names[i],[fig.circle(i,i,color=colors[i],size=20)])]
#fig.add_layout(Legend(items=items),'right')

In [53]:
df1[df1['pair'].isin(['EURUSD','GBPUSD','USDJPY','AUDUSD','USDMXN'])]

,pair,implied_tenor,realised_tenor,quantile,impl_pre,impl_post,imp_chg,bfl_pre,bfl_post,bfl_chg,real,value
0,EURUSD,6M,3M,1,6.234244754464285,6.522676502732241,0.28843174826795615,0.6303487723214286,0.8643461748633878,0.23399740254195922,8.10554918032787,1.8713044258635847
4,EURUSD,6M,3M,5,13.715898442714126,13.011587764182424,-0.7043106785317015,2.0500454949944382,1.8093646273637374,-0.24068086763070085,10.43747274749722,-3.2784256952169066
5,GBPUSD,6M,3M,1,6.959884960718295,7.306957435197817,0.34707247447952216,0.7109822671156004,0.8479139154160982,0.13693164830049775,8.522042428376535,1.5621574676582402
9,GBPUSD,6M,3M,5,13.551002214839423,12.332022369878183,-1.2189798449612397,2.065883388704319,1.7782875968992249,-0.287595791805094,10.432702547065338,-3.1182996677740853
10,USDJPY,6M,3M,1,7.0114821111111105,7.129895903614457,0.11841379250334683,0.8994555555555556,1.0973732530120481,0.19791769745649257,8.525290963855422,1.5138088527443117
14,USDJPY,6M,3M,5,12.419338555555555,12.168630555555556,-0.2507079999999995,2.3996353333333333,2.0201765555555555,-0.3794587777777778,11.430402444444445,-0.9889361111111103
20,AUDUSD,6M,3M,1,8.280873806881244,8.789982978723403,0.5091091718421588,0.7751552719200888,0.9820999999999999,0.20694472807991104,11.489566223404255,3.208692416523011
24,AUDUSD,6M,3M,5,16.425910099889013,15.272494450610433,-1.15341564927858,2.7302830188679246,2.441836847946726,-0.2884461709211985,12.799480799112096,-3.626429300776916
50,USDMXN,6M,3M,1,8.57173,10.9341462585034,2.3624162585033996,1.1949306666666666,1.7413643990929708,0.5464337324263042,13.867930839002266,5.296200839002266
54,USDMXN,6M,3M,5,17.12360975609756,14.956843680709534,-2.1667660753880273,3.6667738359201776,3.1964467849223945,-0.47032705099778305,11.861972283813747,-5.261637472283814


In [65]:
len(pair)

25